In [1]:
import pandas as pd

In [2]:
processed_directory = '../processed_data/'


In [3]:
train_df = pd.read_json('../processed_data/train.json')
test_df = pd.read_json('../processed_data/test.json')

In [4]:
train_df

,id,text,rating,label
0,0,Bromwell High is a cartoon comedy. It ran at t...,9,+
1,10000,Homelessness (or Houselessness as George Carli...,8,+
2,10001,Brilliant over-acting by Lesley Ann Warren. Be...,10,+
3,10002,This is easily the most underrated film inn th...,7,+
4,10003,This is not the typical Mel Brooks film. It wa...,8,+
...,...,...,...,...
24995,9998,"Towards the end of the movie, I felt it was to...",4,-
24996,9999,This is the kind of movie that my enemies cont...,3,-
24997,999,I saw 'Descent' last night at the Stockholm Fi...,3,-
24998,99,Some films that you pick up for a pound turn o...,1,-
